In [1]:
# Most basic stuff for EDA.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Core packages for text processing.

import string
import re

# Libraries for text preprocessing.

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# Loading some sklearn packaces for modelling.

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics import f1_score, accuracy_score

# Some packages for word clouds and NER.

from wordcloud import WordCloud, STOPWORDS
from collections import Counter, defaultdict
from PIL import Image
import spacy
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.5/en_core_web_sm-2.2.5.tar.gz
import en_core_web_sm

# Core packages for general use throughout the notebook.

import random
import warnings
import time
import datetime

# For customizing our plots.

from matplotlib.ticker import MaxNLocator
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches

# Loading pytorch packages.

import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

# Setting some options for general use.

stop = set(stopwords.words('english'))
plt.style.use('fivethirtyeight')
sns.set(font_scale=1.5)
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250
warnings.filterwarnings('ignore')


#Setting seeds for consistent results.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 12.0/12.0 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-py3-none-any.whl size=12011737 sha256=43145819d0f0165eb9d8114588452ba892f973e4b0bd245502635697c2b45b51
  Stored in directory: c:\users\mucle\appdata\local\pip\cache\wheels\57\07\ed\22b6eecc1314cf0e1cbfb45ebeb5ba063dfbc86645f6f23c23
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.4.0
    Uninstalling en-core-web-sm-3.4.0:
      Successfully uninstalled en-core-web-sm-3.4.0


C:\Users\mucle\AppData\Roaming\Python\Python39\site-packages\spacy\util.py:873: UserWarning: [W094] Model 'en_core_web_sm' (2.2.5) specifies an under-constrained spaCy version requirement: >=2.2.2. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.4.1,<3.5.0
  warnings.warn(warn_msg)


In [2]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
# Loading NER.
nlp = en_core_web_sm.load() 

Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-22.2.2-py3-none-any.whl (2.0 MB)


ERROR: To modify pip, please run the following command:
D:\anaconda3\python.exe -m pip install -U pip setuptools wheel


  Using cached setuptools-65.3.0-py3-none-any.whl (1.2 MB)
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 12.8/12.8 MB 5.0 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import pandas as pd
data = pd.read_csv('training_data.csv', on_bad_lines='skip')
datassss=data[100000:101000]

In [4]:
# import module
import geograpy3
from geograpy3 import extraction
from geopy.geocoders import Nominatim
sentence44 = []
sentence48 = [[]]
for index in range(datassss.shape[0]):
    #Latitude = datassss.iloc[index,3].astype(float)
    #Longitude = datassss.iloc[index,4].astype(float)
    #link = 'http://www.bbc.com/news/world-europe-26919928'
    #e = extraction.Extractor(datassss["tweet_text"].to_string())
    #e.find_entities()

    # You can now access all of the places found by the Extractor
    
    #places = geograpy3.get_place_context(url = link)
    text_input = data.iloc[index,:1].to_string()
    places = geograpy3.get_place_context(text = text_input)
    text_input = datassss.iloc[index,:1].to_string()
    more_places = geograpy3.get_place_context(text = text_input)
    sentence48.append(more_places.places)
    print(more_places.places)

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package avera

[]
['Just', 'Thailand']
[]
[]
['Malaysia']
['Twitter']
['EstosSonValoresYPrincipios']
['De']
[]
[]
['Just', 'Ahmedabad', 'India']
['RavaMart']
['Season']
['Turizm']
['RENKA_AIDA', 'ﾊｰｲ']
[]
[]
['MERCADONA']
['Olimpia']
[]
[]
['Gespräch']
[]
['David Spades']
['Argentina']
['Pandemie', 'Impfu']
[]
[]
['Acabou', 'Jardim Morada']
['Gobernador']
[]
[]
['Only Fans']
['TACVerif']
['STOPPLAYIN']
[]
[]
['Hat', 'Allem']
['MLGesnoticia']
['Badass', 'Motorcycle', 'Andranobevava']
['Damn']
[]
['De']
['SIN']
[]
[]
['Damn']
['skywalker_A1983']
['MEGA']
['Gespräch']
['Austin']
[]
['San Franc']
[]
[]
[]
[]
[]
['DjTibzkenya']
[]
['DJ']
['DJ Rigz']
['Sparky Nderu']
['HitsRepublic254']
[]
['هیئت']
[]
['牧羊人派']
[]
[]
[]
['仙台土地高杉くん']
['Socieda']
[]
[]
[]
['Just', 'Malaysia']
['Eu']
[]
['Ciudad', 'Mexico']
['京セラドーム大阪', '大阪市', 'Japon']
[]
['Taco Bell']
[]
[]
['Kabul', 'Militärf']
[]
[]
[]
[]
[]
['广州约炮', '广州线下', '广州兼职', '猜猜多少厘米']
['Entre Ríos']
['金沢魂心家']
[]
[]
['Pandemie', 'Impfu']
[]
[]
[]
['Peru']
[]
['Yahu',

[]
['Congratulations Bangladesh']
[]
[]
[]
['Participamos']
[]
['Rica']
['Pobega']
['Bora', 'Br']
['Miami']
[]
['Just', 'Atlanta', 'United States']
['Sha Allah']
['Büyükçekmece', 'Turchi']
[]
['No']
['十三駅', 'Osaka', 'Japan']
['Lake Forest', 'GBS']
['Меркель']
[]
['Houston']
[]
[]
[]
[]
['Newburg']
['New']
['Just']
['MichelSow Sans']
[]
['大垣駅', '大垣市', 'Japon']
['Perez', 'Madrid']
['Pakistan']
['Sot']
[]
[]
[]
['TilaiyanCKS Nobody']
['Orizaba']
[]
[]
[]
['TPotoms Biden', 'Ghani']
['Sulpan Cave', 'Cold Spring']
['Afghan Refugees', 'Road']
['Dennis Febo']
['Kabul', 'Militärf']
[]
['Sibirien']
['大阪難波駅', '大阪市', 'Japan']
['Chaake']
[]
[]
['Just']
['岡林です']
[]
[]
[]
[]
[]
[]
[]
['Happy']
['BJP4Indore']
['大森海岸駅', '品川区', 'Japon']
['Jakarta', 'Dingin']
[]
[]
[]
['San Joaquín', 'Mackenn']
['Ecuador Cuenca']
[]
['セブンイレブン', '白河インター店', '西郷村', 'Japon']
[]
['Just', 'Smart City']
[]
['Bruno D']
['Mazatlan']
[]
[]
[]
[]
['MichelleYesudas Malaysia']
[]
[]
['金沢区', '横浜市', 'Japon']
['Just', 'City Shah Alam']


In [5]:
df = pd.DataFrame(sentence48)
df

,0,1,2,3
0,None,None,None,None
1,None,None,None,None
2,Just,Thailand,None,None
3,None,None,None,None
4,None,None,None,None
...,...,...,...,...
996,None,None,None,None
997,Turki_alalshikh,None,None,None
998,行ったの,None,None,None
999,Virat Kohli,None,None,None


In [6]:
# render to coordinates


import pandas as pd


from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="spec1")

from geopy.extra.rate_limiter import RateLimiter

from tqdm import tqdm
tqdm.pandas()

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

df['location0'] = df.iloc[:,0].progress_apply(geocode)
df['location1'] = df.iloc[:,1].progress_apply(geocode)
df['location2'] = df.iloc[:,2].progress_apply(geocode)

100%|██████████████████████████████████████████████████████████████████████████████| 1001/1001 [16:49<00:00,  1.01s/it]


In [48]:

df['point0'] = df['location0'].apply(lambda loc: tuple(loc.point) if loc else None)
df['point1'] = df['location1'].apply(lambda loc: tuple(loc.point) if loc else None)
df['point2'] = df['location2'].apply(lambda loc: tuple(loc.point) if loc else None)

In [49]:
df1 = df
ds = datassss

In [50]:
k = ['lat','lon'] 
datassss[k] = datassss[k].apply(pd.to_numeric, errors = 'coerce').dropna() # data type casting and cleaning

In [51]:

df1 = df.fillna(value=np.nan)
df1 = df1.fillna(0)
df1

,0,1,2,3,location0,location1,location2,point0,point1,point2
0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
1,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
2,Just,Thailand,0,0,"(Just, Tęgoborze, gmina Łososina Dolna, powiat...","(ประเทศไทย, (14.8971921, 100.83273))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(49.7239543, 20.6463683, 0.0)","(14.8971921, 100.83273, 0.0)","(44.933143, 7.540121, 0.0)"
3,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
4,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
...,...,...,...,...,...,...,...,...,...,...
996,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
997,Turki_alalshikh,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...",0,"(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
998,行ったの,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...",0,"(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
999,Virat Kohli,0,0,0,"(One8 commune, Delhi-Gurugram Expressway, Mahi...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(28.551561, 77.1225437, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"


In [52]:
ds = ds.fillna(value=np.nan)
ds = ds.fillna(0)
ds

,tweet_text,lang,timestamp,lat,lon
100000,"ゴチ👋 https://t.co/Oq4FtFSSMZ""",ja,2021-08-14T23:01:04.000Z,38.242600,140.362870
100001,Just posted a photo @ Bangkok Thailand https:/...,en,2021-08-19T16:33:57.000Z,13.751450,100.492165
100002,I mean. We all knew this happened https://t.c...,en,2021-08-20T17:24:03.000Z,33.392884,-111.928142
100003,@zenkarasuki ありがとうございます。😃ゝ,ja,2021-08-13T19:14:12.000Z,33.630645,130.548137
100004,Could never imagine the state that Malaysia is...,en,2021-08-20T04:54:53.000Z,3.051220,101.552669
...,...,...,...,...,...
100995,A NY state of mind! Hanging with the girls!!! ...,en,2021-08-07T23:25:49.000Z,40.780709,-73.968542
100996,@Turki_alalshikh \n\nصباح الخير معالي المستشار...,ar,2021-08-15T08:42:13.000Z,24.721943,46.685077
100997,@arsLomt 行ったの?🌊☀\n湘南？,ja,2021-08-10T10:29:44.000Z,35.753959,139.853944
100998,Virat Kohli did the absolutely right thing by ...,en,2021-08-25T17:15:26.000Z,32.328918,75.638853


In [56]:
df1

,0,1,2,3,location0,location1,location2,point0,point1,point2
0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
1,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
2,Just,Thailand,0,0,"(Just, Tęgoborze, gmina Łososina Dolna, powiat...","(ประเทศไทย, (14.8971921, 100.83273))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(49.7239543, 20.6463683, 0.0)","(14.8971921, 100.83273, 0.0)","(44.933143, 7.540121, 0.0)"
3,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
4,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
...,...,...,...,...,...,...,...,...,...,...
996,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
997,Turki_alalshikh,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...",0,"(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
998,行ったの,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...",0,"(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
999,Virat Kohli,0,0,0,"(One8 commune, Delhi-Gurugram Expressway, Mahi...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(28.551561, 77.1225437, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"


In [57]:

df2

,0,1,2,3,location0,location1,location2,point0,point1,point2,class_4,dist1,dist2,dist3
0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",None,4.262834e+06,6.412648e+06,1.231276e+07
1,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",None,4.262834e+06,6.412648e+06,1.231276e+07
2,Just,Thailand,0,0,"(Just, Tęgoborze, gmina Łososina Dolna, powiat...","(ประเทศไทย, (14.8971921, 100.83273))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(49.7239543, 20.6463683, 0.0)","(14.8971921, 100.83273, 0.0)","(44.933143, 7.540121, 0.0)",None,4.262834e+06,6.412648e+06,1.231276e+07
3,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",None,4.262834e+06,6.412648e+06,1.231276e+07
4,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",None,4.262834e+06,6.412648e+06,1.231276e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",None,4.262834e+06,6.412648e+06,1.231276e+07
997,Turki_alalshikh,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...",0,"(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",None,4.262834e+06,6.412648e+06,1.231276e+07
998,行ったの,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...",0,"(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",None,4.262834e+06,6.412648e+06,1.231276e+07
999,Virat Kohli,0,0,0,"(One8 commune, Delhi-Gurugram Expressway, Mahi...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(28.551561, 77.1225437, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",None,4.262834e+06,6.412648e+06,1.231276e+07


In [58]:
df1["class_4"] = None

In [59]:
# Geopy distance module is first imported for computation 
from geopy.distance import geodesic as GD
# Latitude+","+Longitude Next, input the latitude and longitude data for Nairobi and Cairo.  
sentence = []
for index in range(datassss.shape[0]):
    #Latitude1=datassss.iloc[index,3].astype(str)
    #Longitude1 = datassss.iloc[index,4].astype(str)
    X1 = [ds.iloc[index,3].astype(str)+","+ds.iloc[index,4].astype(str)]
    
    
    X2=df1.iloc[index,8]
    #C=df.iloc[96,9]
    X3=df1.iloc[index,9]
    X4 =df1.iloc[index,10] 
    
    if df1.iloc[index,8]!= 0:
        
    # Finally, print the distance between the two locations in kilometers. 
        print("The distance between X1 and X2 is :",GD(X1,X2).m)
        df1['dist1'] = GD(X1,X2).m
    elif df1.iloc[index,9]!= 0:
        print("The distance between X1 and X2 is :",GD(X1,X3).m)
        df1['dist2'] = GD(X1,X3).m
    elif df1.iloc[index,10]!= 0:
        print("The distance between X1 and X2 is :",GD(X1,X4).m) 
        df1['dist3'] = GD(X1,X4).m

The distance between X1 and X2 is : 9653604.872660067
The distance between X1 and X2 is : 9171807.066642655
The distance between X1 and X2 is : 13636054.12179665
The distance between X1 and X2 is : 9582754.680488486
The distance between X1 and X2 is : 10088603.863717694
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 9795534.625810008
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 3130630.887905601
The distance between X1 and X2 is : 7269450.295146299
The distance between X1 and X2 is : 1874809.472495763
The distance between X1 and X2 is : 9858669.351278704
The distance between X1 and X2 is : 8835868.27376412
The distance between X1 and X2 is : 9521428.570119742
The distance between X1 and X2 is : 2823892.652759943
The distance between X1 and X2 is

The distance between X1 and X2 is : 7996008.549911204
The distance between X1 and X2 is : 6572644.709249645
The distance between X1 and X2 is : 9156574.348114971
The distance between X1 and X2 is : 9029929.366885548
The distance between X1 and X2 is : 9503174.770960204
The distance between X1 and X2 is : 6864048.171529858
The distance between X1 and X2 is : 6392859.971532237
The distance between X1 and X2 is : 6484609.670117845
The distance between X1 and X2 is : 8738030.610468911
The distance between X1 and X2 is : 5532196.196402302
The distance between X1 and X2 is : 9697227.8028678
The distance between X1 and X2 is : 14196658.475951927
The distance between X1 and X2 is : 14011684.20373348
The distance between X1 and X2 is : 5728903.613011746
The distance between X1 and X2 is : 991577.968845159
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 7230779.10260643
The distance between X1 and X2 is

The distance between X1 and X2 is : 6653365.518373747
The distance between X1 and X2 is : 14274862.301322537
The distance between X1 and X2 is : 7815531.669033682
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 7887269.819988071
The distance between X1 and X2 is : 5296666.278668398
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 14299748.521695394
The distance between X1 and X2 is : 5787000.419937854
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 10011610.284207579
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 14439770.825702643
The distance between X1 and X2 is : 7469846.53749287
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 14245610.756134903
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 

The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 12382150.695625393
The distance between X1 and X2 is : 7839659.686183373
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 9756609.278672297
The distance between X1 and X2 is : 14015371.483006673
The distance between X1 and X2 is : 14100982.452963218
The distance between X1 and X2 is : 8510110.24310816
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 9706922.771742452
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 14184749.200326646
The distance between X1 and X2 is : 7820825.654605264
The distance between X1 and X2 is 

In [63]:
df2["class_4"] = None
df2 = df2[[0,1,2,3,'location0','location1', 'location2', 'point0', 'point1', 'point2', 'dist1', 'dist2', 'dist3','class_4']]

In [64]:
df2

,0,1,2,3,location0,location1,location2,point0,point1,point2,dist1,dist2,dist3,class_4
0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,None
1,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,None
2,Just,Thailand,0,0,"(Just, Tęgoborze, gmina Łososina Dolna, powiat...","(ประเทศไทย, (14.8971921, 100.83273))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(49.7239543, 20.6463683, 0.0)","(14.8971921, 100.83273, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,None
3,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,None
4,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,None
997,Turki_alalshikh,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...",0,"(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,None
998,行ったの,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...",0,"(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,None
999,Virat Kohli,0,0,0,"(One8 commune, Delhi-Gurugram Expressway, Mahi...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(28.551561, 77.1225437, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,None


In [71]:
df2.iloc[index,11]

6412648.397707576

In [117]:
ds.dtypes

tweet_text     object
lang           object
timestamp      object
lat           float64
lon           float64
dtype: object

In [102]:
# 1-4 range coding 1- lowest precision 4 highest precision of text - coordinate correspondance
for  index in range(df2.shape[0]):    
    if df2.iloc[index,10] < 10 or df2.iloc[index,11]< 10 or df2.iloc[index,12] < 10 :
        df2.iloc[index,13] = 1
    elif 10 < df2.iloc[index,10] < 100 or 10 < df2.iloc[index,11] < 100 or 10 < df2.iloc[index,12] < 100:
        df2.iloc[index,13] = 2
    elif 100 < df2.iloc[index,10] < 1000 or 100 < df2.iloc[index,11] < 1000 or 100 < df2.iloc[index,12] < 1000:
        df2.iloc[index,13] = 3
    else:
        df2.iloc[index,13] = 4 


In [103]:
df2

,0,1,2,3,location0,location1,location2,point0,point1,point2,dist1,dist2,dist3,class_4
0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4
1,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4
2,Just,Thailand,0,0,"(Just, Tęgoborze, gmina Łososina Dolna, powiat...","(ประเทศไทย, (14.8971921, 100.83273))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(49.7239543, 20.6463683, 0.0)","(14.8971921, 100.83273, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4
3,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4
4,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4
997,Turki_alalshikh,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...",0,"(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4
998,行ったの,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...",0,"(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4
999,Virat Kohli,0,0,0,"(One8 commune, Delhi-Gurugram Expressway, Mahi...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(28.551561, 77.1225437, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4


In [126]:
ds

,tweet_text,lang,timestamp,lat,lon
100000,"ゴチ👋 https://t.co/Oq4FtFSSMZ""",ja,2021-08-14T23:01:04.000Z,38.242600,140.362870
100001,Just posted a photo @ Bangkok Thailand https:/...,en,2021-08-19T16:33:57.000Z,13.751450,100.492165
100002,I mean. We all knew this happened https://t.c...,en,2021-08-20T17:24:03.000Z,33.392884,-111.928142
100003,@zenkarasuki ありがとうございます。😃ゝ,ja,2021-08-13T19:14:12.000Z,33.630645,130.548137
100004,Could never imagine the state that Malaysia is...,en,2021-08-20T04:54:53.000Z,3.051220,101.552669
...,...,...,...,...,...
100995,A NY state of mind! Hanging with the girls!!! ...,en,2021-08-07T23:25:49.000Z,40.780709,-73.968542
100996,@Turki_alalshikh \n\nصباح الخير معالي المستشار...,ar,2021-08-15T08:42:13.000Z,24.721943,46.685077
100997,@arsLomt 行ったの?🌊☀\n湘南？,ja,2021-08-10T10:29:44.000Z,35.753959,139.853944
100998,Virat Kohli did the absolutely right thing by ...,en,2021-08-25T17:15:26.000Z,32.328918,75.638853


In [104]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

In [130]:
ds.index = np.arange(1, len(ds) + 1)
ds

,tweet_text,lang,timestamp,lat,lon,index1
1,"ゴチ👋 https://t.co/Oq4FtFSSMZ""",ja,2021-08-14T23:01:04.000Z,38.242600,140.362870,100000
2,Just posted a photo @ Bangkok Thailand https:/...,en,2021-08-19T16:33:57.000Z,13.751450,100.492165,100001
3,I mean. We all knew this happened https://t.c...,en,2021-08-20T17:24:03.000Z,33.392884,-111.928142,100002
4,@zenkarasuki ありがとうございます。😃ゝ,ja,2021-08-13T19:14:12.000Z,33.630645,130.548137,100003
5,Could never imagine the state that Malaysia is...,en,2021-08-20T04:54:53.000Z,3.051220,101.552669,100004
...,...,...,...,...,...,...
996,A NY state of mind! Hanging with the girls!!! ...,en,2021-08-07T23:25:49.000Z,40.780709,-73.968542,100995
997,@Turki_alalshikh \n\nصباح الخير معالي المستشار...,ar,2021-08-15T08:42:13.000Z,24.721943,46.685077,100996
998,@arsLomt 行ったの?🌊☀\n湘南？,ja,2021-08-10T10:29:44.000Z,35.753959,139.853944,100997
999,Virat Kohli did the absolutely right thing by ...,en,2021-08-25T17:15:26.000Z,32.328918,75.638853,100998


In [183]:
#combined =  pd.concat([ds, df2["class_4"]])
combined = pd.concat([ds, df2], axis=1).reindex(ds.index)


In [184]:
combined = pd.concat([ds, df2], axis=1)

In [185]:
combined

,tweet_text,lang,timestamp,lat,lon,index1,0,1,2,3,location0,location1,location2,point0,point1,point2,dist1,dist2,dist3,class_4
1,"ゴチ👋 https://t.co/Oq4FtFSSMZ""",ja,2021-08-14T23:01:04.000Z,38.242600,140.362870,100000.0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4
2,Just posted a photo @ Bangkok Thailand https:/...,en,2021-08-19T16:33:57.000Z,13.751450,100.492165,100001.0,Just,Thailand,0,0,"(Just, Tęgoborze, gmina Łososina Dolna, powiat...","(ประเทศไทย, (14.8971921, 100.83273))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(49.7239543, 20.6463683, 0.0)","(14.8971921, 100.83273, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4
3,I mean. We all knew this happened https://t.c...,en,2021-08-20T17:24:03.000Z,33.392884,-111.928142,100002.0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4
4,@zenkarasuki ありがとうございます。😃ゝ,ja,2021-08-13T19:14:12.000Z,33.630645,130.548137,100003.0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4
5,Could never imagine the state that Malaysia is...,en,2021-08-20T04:54:53.000Z,3.051220,101.552669,100004.0,Malaysia,0,0,0,"(Malaysia, (4.5693754, 102.2656823))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(4.5693754, 102.2656823, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,@Turki_alalshikh \n\nصباح الخير معالي المستشار...,ar,2021-08-15T08:42:13.000Z,24.721943,46.685077,100996.0,Turki_alalshikh,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...",0,"(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4
998,@arsLomt 行ったの?🌊☀\n湘南？,ja,2021-08-10T10:29:44.000Z,35.753959,139.853944,100997.0,行ったの,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...",0,"(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4
999,Virat Kohli did the absolutely right thing by ...,en,2021-08-25T17:15:26.000Z,32.328918,75.638853,100998.0,Virat Kohli,0,0,0,"(One8 commune, Delhi-Gurugram Expressway, Mahi...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(28.551561, 77.1225437, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4
1000,@DanielOmonze @hawtmoe @sendvidbot,und,2021-08-13T08:22:43.000Z,6.625896,3.505413,100999.0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.262834e+06,6.412648e+06,1.231276e+07,4


In [187]:
l0 = ['tweet_text','class_4','location2']
l = ['tweet_text','class_4']

combined = combined[l]
comb = combined

In [188]:
combined

,tweet_text,class_4
1,"ゴチ👋 https://t.co/Oq4FtFSSMZ""",4
2,Just posted a photo @ Bangkok Thailand https:/...,4
3,I mean. We all knew this happened https://t.c...,4
4,@zenkarasuki ありがとうございます。😃ゝ,4
5,Could never imagine the state that Malaysia is...,4
...,...,...
997,@Turki_alalshikh \n\nصباح الخير معالي المستشار...,4
998,@arsLomt 行ったの?🌊☀\n湘南？,4
999,Virat Kohli did the absolutely right thing by ...,4
1000,@DanielOmonze @hawtmoe @sendvidbot,4


In [136]:
#ff = ff[l]
#ff

In [137]:
# BERT odel preparations

In [138]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [139]:
combined = combined.tweet_text.values

In [140]:
# Print the original tweet.

print(' Original: ', combined[0])

# Print the tweet split into tokens.

print('Tokenized: ', tokenizer.tokenize(combined[0]))

# Print the sentence mapped to token ID's.

print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(combined[0])))

 Original:  ゴチ👋 https://t.co/Oq4FtFSSMZ"
Tokenized:  ['[UNK]', 'https', ':', '/', '/', 't', '.', 'co', '/', 'o', '##q', '##4', '##ft', '##fs', '##sm', '##z', '"']
Token IDs:  [100, 16770, 1024, 1013, 1013, 1056, 1012, 2522, 1013, 1051, 4160, 2549, 6199, 10343, 6491, 2480, 1000]


In [141]:
max_len = 0

# For every sentence...

for text in combined:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    
    input_ids = tokenizer.encode(text, add_special_tokens=True)

    # Update the maximum sentence length.
    
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (809 > 512). Running this sequence through the model will result in indexing errors


Max sentence length:  809


In [142]:
# Making list of sentence lenghts:

token_lens = []

for text in combined:
    tokens = tokenizer.encode(text, max_length = 512)
    token_lens.append(len(tokens))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [143]:
# Splitting the train test data after tokenizing.
#labels = combined['class_4'].values
#idx = len(labels)
train= combined[:800]
test = combined[800:]
train.shape

(800,)

In [144]:
def tokenize_map(sentence,labs='None'):
    
    """A function for tokenize all of the sentences and map the tokens to their word IDs."""
    
    global labels
    
    input_ids = []
    attention_masks = []

    # For every sentence...
    
    for text in sentence:
        #   "encode_plus" will:
        
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        
        encoded_dict = tokenizer.encode_plus(
                            text,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            truncation='longest_first', # Activate and control truncation
                            max_length = 400,           # Max length according to our text data.
                            pad_to_max_length = True, # Pad & truncate all sentences.
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the id list. 
        
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    
    if labs != 'None': # Setting this for using this definition for both train and test data so labels won't be a problem in our outputs.
        labels = torch.tensor(labels)
        return input_ids, attention_masks, labels
    else:
        return input_ids, attention_masks
            

In [145]:
labelst = df2['class_4'].values
labels = labelst[0:800]
labels.shape

(800,)

In [146]:
labels= labels.astype(int)
labels

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

In [147]:

input_ids, attention_masks, labels = tokenize_map(train, labels)
test_input_ids, test_attention_masks= tokenize_map(test)

In [148]:
input_ids

tensor([[  101,   100, 16770,  ...,     0,     0,     0],
        [  101,  2074,  6866,  ...,     0,     0,     0],
        [  101,  1045,  2812,  ...,     0,     0,     0],
        ...,
        [  101,  1655, 30197,  ...,     0,     0,     0],
        [  101,  2757,  4442,  ...,     0,     0,     0],
        [  101,  2539,  1024,  ...,     0,     0,     0]])

In [149]:
attention_masks

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [150]:
labels # just 4 class because all 1000 observation are too short

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

In [151]:
# Combine the training inputs into a TensorDataset.

dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 80-20 train-validation split.

# Calculate the number of samples to include in each set.

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

  640 training samples
  160 validation samples


In [152]:
# The DataLoader needs to know our batch size for training, so we specify it here. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32.

batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.

validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [153]:
prediction_data = TensorDataset(test_input_ids, test_attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [154]:
# If there's a GPU available...

if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.  
    
    device = torch.device('cuda')    


    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device('cpu')

No GPU available, using the CPU instead.


In [163]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained(
    'prajjwal1/bert-tiny', #nreimers/BERT-Tiny_L-2_H-128_A-2 https://huggingface.co/models?sort=downloads&search=tiny Use the 124-layer, 1024-hidden, 16-heads, 340M parameters BERT model with an uncased vocab.
    num_labels = 1, # CHANGE TO 4 in the CASE OF NEW TARGET NUBERS FROm 1 to 4 ASIDE FRO 4 only The number of output labels--2 for binary classification. You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the device which we set GPU in our case.

model.to(device)

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, element

In [164]:
# or this model

def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    
    clf_output = sequence_output[:, 0, :]
    
    lay = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    lay = tf.keras.layers.Dense(32, activation='relu')(lay)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    out = tf.keras.layers.Dense(5, activation='softmax')(lay)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [165]:
# Get all of the model's parameters as a list of tuples:

params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print('{:<55} {:>12}'.format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print('{:<55} {:>12}'.format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print('{:<55} {:>12}'.format(p[0], str(tuple(p[1].size()))))


The BERT model has 41 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 128)
bert.embeddings.position_embeddings.weight                (512, 128)
bert.embeddings.token_type_embeddings.weight                (2, 128)
bert.embeddings.LayerNorm.weight                              (128,)
bert.embeddings.LayerNorm.bias                                (128,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (128, 128)
bert.encoder.layer.0.attention.self.query.bias                (128,)
bert.encoder.layer.0.attention.self.key.weight            (128, 128)
bert.encoder.layer.0.attention.self.key.bias                  (128,)
bert.encoder.layer.0.attention.self.value.weight          (128, 128)
bert.encoder.layer.0.attention.self.value.bias                (128,)
bert.encoder.layer.0.attention.output.dense.weight        (128, 128)
bert.encoder.layer.0.attention.output.dense.bias              (1

In [166]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch).

# The 'W' stands for 'Weight Decay fix' probably...

optimizer = AdamW(model.parameters(),
                  lr = 6e-6, # args.learning_rate
                  eps = 1e-8 # args.adam_epsilon
                )

In [167]:
# Number of training epochs. The BERT authors recommend between 2 and 4. 

# We chose to run for 3, but we'll see later that this may be over-fitting the training data.

epochs = 2

# Total number of training steps is [number of batches] x [number of epochs] (Note that this is not the same as the number of training samples).
total_steps = (len(train_dataloader) * epochs)

# Create the learning rate scheduler.

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [168]:
def flat_accuracy(preds, labels):
    
    """A function for calculating accuracy scores"""
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    return accuracy_score(labels_flat, pred_flat)

def flat_f1(preds, labels):
    
    """A function for calculating f1 scores"""
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    return f1_score(labels_flat, pred_flat)

In [169]:
def format_time(elapsed):    
    
    """A function that takes a time in seconds and returns a string hh:mm:ss"""
    
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [171]:
# second variant for non working in the case of attribute ite error

training_stats = []

# Measure the total training time for the whole run.

total_t0 = time.time()

# For each epoch...

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print('')
    print()
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes:
    
    t0 = time.time()

    # Reset the total loss for this epoch.
    
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to `train` just changes the *mode*, it doesn't *perform* the training.
    
    # `dropout` and `batchnorm` layers behave differently during training vs. test ,
    # source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch
    
    model.train()

    # For each batch of training data...
    
    for step, batch in enumerate(train_dataloader):

        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the device(gpu in our case) using the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        
        b_input_ids = batch[0].to(device).to(torch.int64)
        b_input_mask = batch[1].to(device).to(torch.int64)
        b_labels = batch[2].to(device).to(torch.int64)

        # Always clear any previously calculated gradients before performing a backward pass. PyTorch doesn't do this automatically because accumulating the gradients is 'convenient while training RNNs'. 
        # Source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
        
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is down here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers BertForSequenceClassification.
        
        # It returns different numbers of parameters depending on what arguments given and what flags are set. For our useage here, it returns the loss (because we provided labels),
        # And the 'logits' (the model outputs prior to activation.)
        
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can calculate the average loss at the end, 
        # `loss` is a tensor containing a single value; the `.item()` function just returns the Python value from the tensor.
        
        #total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        
        #loss.backward()

        # Clip the norm of the gradients to 1.0 This is to help prevent the 'exploding gradients' problem.
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        
        # The optimizer dictates the 'update rule'(How the parameters are modified based on their gradients, the learning rate, etc.)
        
        optimizer.step()

        # Update the learning rate.
        
        scheduler.step()

    # Calculate the average loss over all of the batches.
    
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    
    training_time = format_time(time.time() - t0)

    print('')
    print('  Average training loss: {0:.2f}'.format(avg_train_loss))
    print('  Training epcoh took: {:}'.format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on our validation set.

    print('')
    print('Running Validation...')

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently during evaluation.
    
    model.eval()

    # Tracking variables:
    
    total_eval_accuracy = 0
    total_eval_loss = 0
    total_eval_f1 = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch.
    
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        
        # As we unpack the batch, we'll also copy each tensor to the GPU using the `to` method.
        
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during the forward pass, since this is only needed for backprop (training part).
        
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the 'segment ids', which differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is down here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers BertForSequenceClassification.
            # Get the 'logits' output by the model. The 'logits' are the output values prior to applying an activation function like the softmax.
            
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels,return_dict = False)
            
        # Accumulate the validation loss.
        
        #total_eval_loss += loss.item()

        # Move logits and labels to CPU:
        
        #logits = logits.detach().cpu().numpy()
        #label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and accumulate it over all batches:
        
        #total_eval_accuracy += flat_accuracy(logits, label_ids)
        #total_eval_f1 += flat_f1(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    
    #avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    #print('  Accuracy: {0:.2f}'.format(avg_val_accuracy))
    
    # Report the final f1 score for this validation run.
    
    #avg_val_f1 = total_eval_f1 / len(validation_dataloader)
    #print('  F1: {0:.2f}'.format(avg_val_f1))

    # Calculate the average loss over all of the batches.
    
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    
    
    # Measure how long the validation run took:
    
    validation_time = format_time(time.time() - t0)
    
    print('  Validation Loss: {0:.2f}'.format(avg_val_loss))
    print('  Validation took: {:}'.format(validation_time))

    # Record all statistics from this epoch.
    
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            #'Valid. Accur.': avg_val_accuracy,
            #'Val_F1' : avg_val_f1,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print('')
print('Training complete!')

print('Total training took {:} (h:mm:ss)'.format(format_time(time.time()-total_t0)))



======== Epoch 1 / 2 ========
Training...

  Average training loss: 0.00
  Training epcoh took: 0:00:13

Running Validation...
  Validation Loss: 0.00
  Validation took: 0:00:02


======== Epoch 2 / 2 ========
Training...

  Average training loss: 0.00
  Training epcoh took: 0:00:11

Running Validation...
  Validation Loss: 0.00
  Validation took: 0:00:02

Training complete!
Total training took 0:00:28 (h:mm:ss)


In [174]:
# Prediction on test set:

print('Predicting labels for {:,} test sentences...'.format(len(test_input_ids)))

# Put model in evaluation mode:

model.eval()

# Tracking variables :

predictions = []

# Predict:

for batch in prediction_dataloader:
    
  # Add batch to GPU

    batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader:
    
    b_input_ids, b_input_mask, = batch
  
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction:

    with torch.no_grad():
      # Forward pass, calculate logit predictions:
    
          outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

    logits = outputs[0]

  # Move logits and labels to CPU:
    
    logits = logits.detach().cpu().numpy()
 
  
  # Store predictions and true labels:
    
    predictions.append(logits)


print('    DONE.')

Predicting labels for 200 test sentences...
    DONE.


In [175]:
# Getting list of predictions and then choosing the target value with using argmax on probabilities.

flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

In [182]:
combined

array(['ゴチ👋 https://t.co/Oq4FtFSSMZ"',
       'Just posted a photo @ Bangkok Thailand https://t.co/5rKm0pLHeY',
       'I mean.  We all knew this happened https://t.co/oOw4roOPAW',
       '@zenkarasuki ありがとうございます。😃ゝ',
       'Could never imagine the state that Malaysia is in now.. Sad merdeka when democracy fails.. https://t.co/gCqUDFnoSd',
       'I never post on Twitter ,Etats-Unis,en,2021-08-04T16:07:48.000Z,33.5422375\n@smallboy_xs https://t.co/dq3gnP1Rhu,und,2021-08-04T09:57:50.000Z,16.753031,100.2061691\n#LaVisionEsUnAprendizaje',
       '#EstosSonValoresYPrincipios que se violentan a diario!!!',
       'De seguro no irá a cumplir con los compromisos que el #PresidenteDeMexico @lopezobrador_ y el pueblo de México le han encomendado ???',
       'Cómo siempre', '#Caricaturas58 https://t.co/4sYJRZFidb"',
       'Just posted a photo @ Ahmedabad,India,en,2021-08-01T16:02:28.000Z,23.01395875\nPouah vive Strasbourg hin 😭😭 https://t.co/IHiyxLQmFs,fr,2021-08-31T22:55:36.000Z,48.8387174,7

In [189]:
submission = comb[800:1000]
submission['target'] = flat_predictions
submission.head(10)

,tweet_text,class_4,target
801,#healedtattoo en Ecuador Cuenca https://t.co/Y...,4,0
802,https://t.co/0H7FaxExbR,4,0
803,"I'm at セブンイレブン 白河インター店 - @711sej in 西郷村,Japon,...",4,0
804,preciso de um psicólogo,4,0
805,Just posted a photo @ Bhubaneswar - Smart City...,4,0
806,Gu nước ngoài https://t.co/AVY68GaFom,4,0
807,@randolfeap \nPorquê não convocar o Sr Bruno D...,4,0
808,Alguna pareja en Mazatlan (hetero) que me deje...,4,0
809,Now I'm made I didn't move to NY like I wanted...,4,0
810,@Nandocaquariana Acredito que vai passar todo ...,4,0


In [190]:
y_true = submission['class_4'].to_numpy().astype(float)
y_pred = submission['target'].to_numpy().astype(float)

In [191]:
y_true

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.])

In [192]:
f1_score(y_true, y_pred, average=None)

array([0., 0.])

In [193]:
f1_score(y_true, y_pred, average='macro')

0.0

In [194]:
f1_score(y_true, y_pred, average='weighted')

0.0

In [195]:
f1_score(y_true, y_pred, average='micro')

0.0